# 2D Convolution With Stride & Padding

In [96]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os
import cv2

In [97]:
import tensorflow as tf

### Reading data as image and resizing and test, train & validation separated

In [98]:
def read_data(path):
    
    train_path = path+"/train"
    test_path = path+"/test"
    validation_path = path+"/val"
    
    tr_data, test_data, val_data = [], [], []
    tr_out, test_out, val_out = [], [], []

    for i in os.listdir(train_path):

        try:
            for j in os.listdir(train_path+"/"+i):
                image = cv2.imread(train_path+"/"+i+"/"+j, cv2.IMREAD_GRAYSCALE)
                tr_data.append(cv2.resize(image, (224, 224)))
                tr_out.append(i)

            for j in os.listdir(test_path+"/"+i):
                image = cv2.imread(test_path+"/"+i+"/"+j, cv2.IMREAD_GRAYSCALE)
                test_data.append(cv2.resize(image, (224, 224)))
                test_out.append(i)

            for j in os.listdir(validation_path+"/"+i):
                image = cv2.imread(validation_path+"/"+i+"/"+j, cv2.IMREAD_GRAYSCALE)
                val_data.append(cv2.resize(image, (224, 224)))
                val_out.append(i)
        except:
            pass
                
                
    tr_data, test_data, val_data = np.array(tr_data), np.array(test_data), np.array(val_data)
    tr_out, test_out, val_out = np.array(tr_out), np.array(test_out), np.array(val_out)

    return tr_data, test_data, val_data, tr_out, test_out, val_out



In [99]:
path = "/Users/rajeshr/Desktop/Group_22"
tr_data, test_data, val_data, tr_out, test_out, val_out = read_data(path)

## Kaiming Initializatied Filter

In [100]:
def kaiming(n, size):
    mean, sd = 0, np.sqrt(2/n)
    weights = np.random.normal(mean, sd, size=size)
    return weights

In [101]:
np.random.seed(0)
n = 1
filtr = kaiming(n, (3, 3))
filtr

array([[ 2.49474675,  0.56590775,  1.38414453],
       [ 3.16910155,  2.64112584, -1.38207963],
       [ 1.34362793, -0.21405142, -0.1459735 ]])

## Convolutional Filter

In [102]:
def zero_padding(inpt, padding):
    
    m, n = inpt.shape
    
    temp = []
    for i in range(padding):
        for j in range(m+2*padding):
            temp.append(0)
            
    for i in inpt:
        
        for j in range(padding):
            temp.append(0)
        
        for p in i:
            temp.append(p)
            
        for k in range(padding):
            temp.append(0)
            
    for i in range(padding):
        for j in range(m+2*padding):
            temp.append(0)
    
    
    temp = np.array(temp, dtype=np.uint8)
    temp = temp.flatten()
    temp = temp.reshape(m+2*padding, n+2*padding)
    
    return temp

In [105]:
def convolution(inpt, filtr , stride, padding):
    
    (Am, An), (fm, fn) = inpt.shape, filtr.shape
    inpt = zero_padding(inpt, padding)
    m, n = inpt.shape
    
    feature_map = []
    
    flip_leftright = np.fliplr(filtr)
    flip_updown = np.flipud(flip_leftright)
    Filtr = flip_updown 
    
    Outm, Outn = int(((Am-fm+2*padding)/stride)+1), int(((An-fn+2*padding)/stride)+1)
    
    feature_map = []
    for i in range(0, m, stride):
        for j in range(0, n, stride):
            if inpt[i:i+fn, j:j+fm].shape == Filtr.shape:
                feature_map.append((Filtr*inpt[i:i+fn, j:j+fm]).sum())

    feature_map = np.array(feature_map).reshape(Outm, Outn)
    return feature_map

In [106]:
feature_map = convolution(tr_data[0], filtr, stride = 1, padding = 0)
print(feature_map)
cv2.imwrite('/Users/rajeshr/Desktop/feature_map'+str(np.random.randint(1000))+'.jpg', feature_map)
tr_data[0].shape, feature_map.shape

[[ 118.04355169  138.13555937  179.71439218 ...  184.67655187
   185.74432824  187.27444627]
 [ 139.5091461   188.29690382  216.01093463 ...  181.8274783
   185.88823684  187.27444627]
 [ 185.90084279  219.94093638  211.78896215 ...  182.87850929
   188.8004345   187.27444627]
 ...
 [ 685.76281864  545.49490864  526.28045466 ... 1416.83870728
  1252.37647778 1203.79635386]
 [ 703.8072804   606.26447599  600.74739042 ... 1400.21484511
  1276.66501691 1266.04384796]
 [ 698.03999022  634.65098557  641.96535066 ... 1355.52774545
  1383.40209245 1352.74094928]]


((224, 224), (222, 222))